In [58]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.metrics import accuracy_score
from lightgbm import LGBMRegressor
from lightgbm import LGBMClassifier
import gc
from sklearn.preprocessing import normalize
import torch
from sklearn.decomposition import TruncatedSVD
from sklearn.svm import NuSVR,SVR
from collections import Counter
from sklearn.metrics import cohen_kappa_score
import xgboost
from sklearn.neural_network import MLPRegressor


In [59]:
# !/home/kaor/.conda/envs/mmdet/bin/python -m pip install xgboost

In [70]:
def MCRMSE(y_trues, y_preds):
    scores = []
    idxes = y_trues.shape[1]
    for i in range(idxes):
        y_true = y_trues[:,i]
        y_pred = y_preds[:,i]
        score = mean_squared_error(y_true, y_pred, squared=False) # RMSE
        scores.append(score)
    mcrmse_score = np.mean(scores)
    return mcrmse_score, scores

def get_score(y_trues, y_preds):
    mcrmse_score, scores = MCRMSE(y_trues, y_preds)
    return mcrmse_score, scores
def get_data(state,dim=128):
    X_train,y_train,X_val,y_val,XX_train,XX_val = state['EMB_train'],state['train_labels'],state['EMB'],state['valid_labels'],state['EMB_train_real'],state['EMB_real']
    X = np.concatenate([X_train,X_val])
    svd = TruncatedSVD(n_components=dim)
    EMB = svd.fit_transform(X)
    X_train_svd = EMB[:len(X_train)]
    X_val_svd = EMB[len(X_train):]
    return X_train,y_train,X_val,y_val,X_train_svd,X_val_svd,XX_train,XX_val
def train_lbg_reg(X_train_svd,y_train,X_val_svd,y_val):
    model = LGBMRegressor(metric = 'rmse', learning_rate=0.1,n_estimators =1000,reg_lambda = 0.5,
                          max_depth=7,num_leaves=13,bagging_fraction = 0.6, feature_fraction = 0.7,)# NuSVR()#xgboost.XGBRegressor(n_estimators=1000, max_depth=7, eta=0.1, subsample=0.7, colsample_bytree=0.8)

    model.fit(X_train_svd, y_train,eval_set=(X_val_svd, y_val),verbose=250,early_stopping_rounds=20)
    pred = model.predict(X_val_svd)
    return pred
def train_lbg_cla(X_train_svd,y_train,X_val_svd,y_val):
    w = np.array([np.count_nonzero(y_train.round().astype(int) == row) for row in range(5)])
    class_weight = {0:w[0],1:w[1],2:w[2],4:w[3]*100,4:w[4]*10}
    model = LGBMClassifier(learning_rate=0.1,n_estimators =300,reg_lambda = 0.5,class_weight = class_weight)
    model.fit(X_train_svd, y_train.round().astype(int),eval_set=(X_val_svd, y_val.round().astype(int)),verbose=250,early_stopping_rounds=20)
    pred = model.predict(X_val_svd)
    return pred

In [38]:
# model = xgboost.XGBRegressor()
# model.fit?

In [3]:
!ls outputs_toxigen_backtranslate_reg/Narrativaai.deberta-v3-small-finetuned-hate_speech18

config.pth
Narrativaai-deberta-v3-small-finetuned-hate_speech18_fold0_best.pth
Narrativaai-deberta-v3-small-finetuned-hate_speech18_fold0_pre.pth
Narrativaai-deberta-v3-small-finetuned-hate_speech18_fold1_best.pth
Narrativaai-deberta-v3-small-finetuned-hate_speech18_fold2_best.pth
Narrativaai-deberta-v3-small-finetuned-hate_speech18_fold3_best.pth
oof_df.pkl
tokenizer
train.log


In [48]:
import glob
root = 'outputs_toxigen_backtranslate_reg'
all_pth = glob.glob('outputs_toxigen_backtranslate_reg/*/*best.pth')
FOLDS = 4

In [5]:
all_pth

['outputs_toxigen_backtranslate_reg/Narrativaai.deberta-v3-small-finetuned-hate_speech18/Narrativaai-deberta-v3-small-finetuned-hate_speech18_fold0_best.pth',
 'outputs_toxigen_backtranslate_reg/Narrativaai.deberta-v3-small-finetuned-hate_speech18/Narrativaai-deberta-v3-small-finetuned-hate_speech18_fold1_best.pth',
 'outputs_toxigen_backtranslate_reg/Narrativaai.deberta-v3-small-finetuned-hate_speech18/Narrativaai-deberta-v3-small-finetuned-hate_speech18_fold2_best.pth',
 'outputs_toxigen_backtranslate_reg/Narrativaai.deberta-v3-small-finetuned-hate_speech18/Narrativaai-deberta-v3-small-finetuned-hate_speech18_fold3_best.pth',
 'outputs_toxigen_backtranslate_reg/microsoft.deberta-v3-base/microsoft-deberta-v3-base_fold0_best.pth',
 'outputs_toxigen_backtranslate_reg/microsoft.deberta-v3-base/microsoft-deberta-v3-base_fold1_best.pth']

In [6]:
# !ls outputs_toxigen_backtranslate_reg/Narrativaai.deberta-v3-small-finetuned-hate_speech18/

In [7]:
state = torch.load(all_pth[0])
state.keys()

dict_keys(['model', 'predictions', 'EMB_train_real', 'EMB_real', 'EMB', 'valid_labels', 'EMB_train', 'valid_folds', 'train_labels', 'train_folds'])

# Train Regressor On Embeddings from pre-trin

In [9]:
output_reg_real = {}
pred_reg = []
val_reg = []
for fold in range(FOLDS):
    all_pth_fold = [row for row in all_pth if f"_fold{fold}" in row]
    print(len(all_pth_fold))
    XXX_train = []
    XXX_val = []
    for p in all_pth_fold:
        state = torch.load(p)
        X_train,y_train,X_val,y_val,X_train_svd,X_val_svd,XX_train,XX_val = get_data(state)
        XXX_train.append(XX_train)
        XXX_val.append(XX_val)
    XXX_train = np.array(XXX_train).transpose(1,2,0)
    XXX_train = XXX_train.reshape(XXX_train.shape[0],-1)
    XXX_val = np.array(XXX_val).transpose(1,2,0)
    XXX_val = XXX_val.reshape(XXX_val.shape[0],-1)
    pred_lbg_reg = train_lbg_reg(XXX_train,y_train.ravel(),XXX_val,y_val.ravel())
    score_lbg_reg = get_score(y_val.reshape(-1,1),pred_lbg_reg.reshape(-1,1))
    output_reg_real[f'fold_{fold}_score'] = score_lbg_reg[0]
    pred_reg = pred_reg + list(pred_lbg_reg.reshape(-1))
    val_reg = val_reg + list(y_val)
    break
cv = get_score(np.array(val_reg).reshape(-1,1),np.array(pred_reg).reshape(-1,1))
output_reg_real['cv_score'] = cv[0]
output_reg_real

2


/home/kaor/.local/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/kaor/.local/lib/python3.7/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


{'fold_0_score': 0.21868971226322728, 'cv_score': 0.21868971226322728}

# Train Regressor On Embeddings After fine tune

In [ ]:
output_reg = {}
pred_reg = []
val_reg = []
for fold in range(FOLDS):
    all_pth_fold = [row for row in all_pth if f"_fold{fold}" in row]
    print(len(all_pth_fold))
    XXX_train = None
    XXX_val = None
    for p in all_pth_fold:
        state = torch.load(p)
        dim = 128 if "deberta-v3-base" in p else 96
        X_train,y_train,X_val,y_val,X_train_svd,X_val_svd,XX_train,XX_val = get_data(state,dim)
        if XXX_train is None:
            XXX_train = X_train_svd
            XXX_val = X_val_svd
        else:
            XXX_train = np.concatenate([XXX_train,X_train_svd],axis=1)
            XXX_val = np.concatenate([XXX_val,X_val_svd],axis=1)
    XXX_train = normalize(XXX_train)
    XXX_val = normalize(XXX_val)
    pred_lbg_reg = train_lbg_reg(XXX_train,y_train.ravel(),XXX_val,y_val.ravel())
    score_lbg_reg = get_score(y_val.reshape(-1,1),pred_lbg_reg.reshape(-1,1))
    output_reg[f'fold_{fold}_score'] = score_lbg_reg[0]
    pred_reg = pred_reg + list(pred_lbg_reg.reshape(-1))
    val_reg = val_reg + list(y_val)
    # if fold == 0:
    #     break
cv = get_score(np.array(val_reg).reshape(-1,1),np.array(pred_reg).reshape(-1,1))
output_reg['cv_score'] = cv[0]
output_reg

2


/home/kaor/.local/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/kaor/.local/lib/python3.7/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


2


/home/kaor/.local/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/kaor/.local/lib/python3.7/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


2


# Train Classifier On Embeddings from pre-trin

In [49]:
output_cla_real = {}
pred_cla = []
val_cla = []
for fold in range(FOLDS):
    state = torch.load(f"{path}/{model}_fold{fold}_best.pth")
    X_train,y_train,X_val,y_val,X_train_svd,X_val_svd,XX,XX_val = get_data(state)
    y_val = y_val*4
    y_train = y_train*4
    pred_lbg_cla = train_lbg_cla(XX,y_train.ravel().round().astype(int),XX_val,y_val.ravel().round().astype(int))
    score_lbg_cla = accuracy_score(y_val.round().astype(int),pred_lbg_cla)
    output_cla_real[f'fold_{fold}_score'] = score_lbg_cla
    val_cla = val_cla + list(y_val)
    pred_cla = pred_cla + list(pred_lbg_cla.reshape(-1))
cv = accuracy_score(np.array(val_cla).round().astype(int),np.array(pred_cla))
output_cla_real['cv_score_acc'] = cv
output_cla_real['cv_score_kappa']  = cohen_kappa_score(np.array(val_cla).round().astype(int),np.array(pred_cla))
cm = confusion_matrix(np.array(val_cla).round().astype(int)>2,np.array(pred_cla)>1)
cmn = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(cmn, annot=True, fmt='.2f')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show(block=False)
cm = confusion_matrix(np.array(val_cla).round().astype(int),np.array(pred_cla))
cmn = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(cmn, annot=True, fmt='.2f')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show(block=False)
output_cla_real

NameError: name 'path' is not defined

# Train Classifier On Embeddings After fine tune

In [ ]:
output_cla = {}
pred_cla = []
val_cla = []
for fold in range(FOLDS):
    state = torch.load(f"{path}/{model}_fold{fold}_best.pth")
    X_train,y_train,X_val,y_val,X_train_svd,X_val_svd,XX,XX_val = get_data(state)
    y_val = y_val*4
    y_train = y_train*4
    pred_lbg_cla = train_lbg_cla(X_train_svd,y_train.ravel().round().astype(int),X_val_svd,y_val.ravel().round().astype(int))
    score_lbg_cla = accuracy_score(y_val.round().astype(int),pred_lbg_cla)
    output_cla[f'fold_{fold}_score'] = score_lbg_cla
    val_cla = val_cla + list(y_val)
    pred_cla = pred_cla + list(pred_lbg_cla.reshape(-1))
cv = accuracy_score(np.array(val_cla).round().astype(int),np.array(pred_cla))
output_cla['cv_score_acc'] = cv
output_cla['cv_score_kappa']  = cohen_kappa_score(np.array(val_cla).round().astype(int),np.array(pred_cla))
cm = confusion_matrix(np.array(val_cla).round().astype(int)>2,np.array(pred_cla)>1)
cmn = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(cmn, annot=True, fmt='.2f')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show(block=False)
cm = confusion_matrix(np.array(val_cla).round().astype(int),np.array(pred_cla))
cmn = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(cmn, annot=True, fmt='.2f')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show(block=False)
output_cla

/home/kaor/.local/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/kaor/.local/lib/python3.7/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
/home/kaor/.local/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds'

In [ ]:
output_reg_real,output_reg

In [ ]:
output_cla_real,output_cla